<a href="https://colab.research.google.com/github/metinuslu/blm5103_cv_project/blob/main/instance_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# define the possible values for genre
genres = ["action", "comedy", "drama", "horror", "romance"]

# generate 1000 random ids
ids = np.arange(1, 1001)
# generate 1000 random filenames
filenames = np.array([f"file_{id}.txt" for id in ids])
# generate a 1000 x 1024 array of random numbers between 0 and 255
x = np.random.randint(0, 256, size=(1000, 1024))
# choose 1000 random genres from the list
genres = np.random.choice(genres, size=1000)

# create a dataframe from the arrays
df = pd.DataFrame(np.column_stack([ids, filenames, x, genres]))
# rename the columns
df.columns = ["id", "filename"] + [f"x{i}" for i in range(1, 1025)] + ["genre"]

# write the dataframe to a csv file
df.to_csv("records.csv", index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/blm5103_cv_project/records.csv")

In [ ]:
df.head()

,id,filename,x1,x2,x3,x4,x5,x6,x7,x8,...,x1016,x1017,x1018,x1019,x1020,x1021,x1022,x1023,x1024,genre
0,1,file_1.txt,102,94,75,20,26,57,248,227,...,19,116,165,81,20,253,97,18,92,drama
1,2,file_2.txt,51,216,214,43,248,55,196,215,...,242,103,128,172,221,55,110,115,241,romance
2,3,file_3.txt,86,44,250,79,134,96,87,132,...,116,19,250,5,54,95,94,152,2,horror
3,4,file_4.txt,41,123,70,252,135,132,2,94,...,224,194,201,132,100,111,159,162,175,action
4,5,file_5.txt,123,11,157,158,145,21,84,69,...,191,224,62,231,174,35,51,19,236,action


In [ ]:
from torchvision import models
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torchvision.transforms as transforms

In [ ]:
encoder = LabelEncoder()

In [ ]:
df['encoded_target'] = encoder.fit_transform(df['genre'])

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __repr__(self):
      return "CustomDataset"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # İlgili özellikleri ve etiketi seçme
        features = torch.tensor(self.data.iloc[idx, :-1], dtype=torch.float)
        target = torch.tensor(self.data.iloc[idx, -1], dtype=torch.long)

        return features, target

In [ ]:
filtred_df = df.loc[:, [col for col in df.columns if col not in ("filename", "id", "genre")]]

In [ ]:
dataset = CustomDataset(df.loc[:, [col for col in df.columns if col not in ("filename", "id", "genre")]])

In [ ]:
dataset

CustomDataset

In [ ]:
train_data, val_data = train_test_split(filtred_df, test_size=0.2, random_state=42)

In [ ]:
train_data.loc[:, train_data.columns != "encoded_target"]

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x1015,x1016,x1017,x1018,x1019,x1020,x1021,x1022,x1023,x1024
29,155,113,120,143,181,148,29,90,155,205,...,198,212,157,163,177,53,82,196,83,130
535,160,221,128,165,16,10,254,89,235,120,...,86,72,172,143,157,205,43,185,6,139
695,233,179,14,43,247,222,159,36,143,237,...,46,122,48,231,230,7,12,153,186,46
557,33,221,217,39,224,221,94,222,168,11,...,93,182,205,109,239,102,204,166,193,112
836,245,156,101,198,3,193,31,233,91,136,...,112,25,109,68,234,140,192,27,211,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,46,219,227,252,140,186,43,124,145,209,...,212,23,93,117,150,41,118,224,60,59
270,100,243,58,179,114,196,139,138,96,133,...,38,243,167,138,59,172,48,140,85,23
860,107,77,235,157,255,177,142,141,59,42,...,146,183,61,179,223,3,250,235,240,244
435,225,151,78,132,160,205,148,132,7,217,...,163,30,105,238,76,221,10,18,44,205


In [ ]:
X_train = torch.tensor(train_data.loc[:, train_data.columns != "encoded_target"].values, dtype=torch.float)

In [ ]:
y_train = torch.tensor(train_data.loc[:, train_data.columns == "encoded_target"].values, dtype=torch.float)

In [ ]:
y_train

In [ ]:
X_train.view(800 ,32, 32)

In [ ]:
# DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [ ]:
# Load the pretrained model
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_ftrs = model.fc.in_features

# Replace the fully connected layer for transfer learning
model.fc = nn.Linear(num_ftrs, 4)

In [ ]:
s = X_train[1].view(3 ,32, 32)

In [ ]:
X_train[1]

tensor([160., 221., 128.,  ..., 185.,   6., 139.])

In [ ]:
model(s)

RuntimeError: ignored

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        inputs = inputs.view(-1,1 ,32, 32)
        print(inputs.shape)
        outputs = modified_resnet(inputs)
        loss = criterion(outputs, targets.reshape(-1, 1, 1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print average loss per epoch
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {running_loss / len(train_loader)}")

In [ ]:
# Replace the last layer with a new classifier
num_classes = 4 # assuming 4 classes in your dataset
model.fc = nn.Sequential(
    nn.Linear(1 , num_classes), # create a new linear layer with 5 outputs
    nn.Softmax(dim = 1)
)

In [ ]:
modified_resnet = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
    *list(model.children())[1:]  # Use the rest of the layers from the pre-trained ResNet
)

In [ ]:
if isinstance(model, models.ResNet):
    modified_resnet[0].weight.data = torch.mean(model.conv1.weight, dim=1, keepdim=True)

In [ ]:
modified_resnet[9]

Sequential(
  (0): Linear(in_features=1, out_features=4, bias=True)
  (1): Softmax(dim=1)
)

In [ ]:
for param in modified_resnet.parameters(): # freeze every layer
    param.requires_grad = False

In [ ]:
for param in modified_resnet[9].parameters(): # except last fully connected layer
    param.requires_grad = True

In [ ]:
# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
modified_resnet

In [ ]:
num_epochs = 10

In [ ]:
for param in resnet.parameters():
    param.requires_grad = False

In [ ]:
# Define a function to trace tensor shapes through the model
def trace_shapes(model, x):
    for name, layer in resnet.named_children():
        x = layer(x)
        print(f"Layer: {name}, Output shape: {x.shape}")

In [ ]:
sample_input.shape

torch.Size([32, 3, 224, 224])

In [ ]:
trace_shapes(resnet, sample_input)

Layer: conv1, Output shape: torch.Size([32, 64, 112, 112])
Layer: bn1, Output shape: torch.Size([32, 64, 112, 112])
Layer: relu, Output shape: torch.Size([32, 64, 112, 112])
Layer: maxpool, Output shape: torch.Size([32, 64, 56, 56])
Layer: layer1, Output shape: torch.Size([32, 64, 56, 56])
Layer: layer2, Output shape: torch.Size([32, 128, 28, 28])
Layer: layer3, Output shape: torch.Size([32, 256, 14, 14])
Layer: layer4, Output shape: torch.Size([32, 512, 7, 7])
Layer: avgpool, Output shape: torch.Size([32, 512, 1, 1])


RuntimeError: ignored

In [ ]:
trace_shapes(resnet, sample)

Layer: 0, Output shape: torch.Size([32, 64, 16, 16])
Layer: 1, Output shape: torch.Size([32, 64, 16, 16])
Layer: 2, Output shape: torch.Size([32, 64, 16, 16])
Layer: 3, Output shape: torch.Size([32, 64, 8, 8])
Layer: 4, Output shape: torch.Size([32, 256, 8, 8])
Layer: 5, Output shape: torch.Size([32, 512, 4, 4])
Layer: 6, Output shape: torch.Size([32, 1024, 2, 2])
Layer: 7, Output shape: torch.Size([32, 2048, 1, 1])
Layer: 8, Output shape: torch.Size([32, 2048, 1, 1])
Layer: 9, Output shape: torch.Size([32, 2048, 1, 4])


In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        inputs = inputs.view(-1,1 ,32, 32)
        print(inputs.shape)
        outputs = modified_resnet(inputs)
        loss = criterion(outputs, targets.reshape(-1, 1, 1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print average loss per epoch
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {running_loss / len(train_loader)}")

In [ ]:
inputs = inputs.view(-1, 1, 32, 32)

In [ ]:
outputs.shape

torch.Size([32, 2048, 1, 4])

In [ ]:
sample_input = torch.randn(32, 3, 224, 224)

In [ ]:
# Load a pre-trained ResNet-18 model
resnet = models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
output = resnet(sample_input)

In [ ]:
output

tensor([[-2.2982e+00, -2.4732e+00,  3.1293e-01,  ..., -2.5533e+00,
          6.3689e-01,  5.1853e-01],
        [-1.1307e+00, -1.4908e-01, -9.3917e-01,  ...,  6.7538e-04,
          9.3072e-01,  1.8026e+00],
        [ 1.1939e-01, -9.7078e-01, -9.6343e-01,  ..., -1.7808e+00,
          2.2929e+00, -8.9892e-01],
        ...,
        [-7.8178e-01, -1.3978e+00,  2.6497e+00,  ..., -8.8405e-01,
          3.1901e-01,  5.6778e-01],
        [-8.1442e-01, -1.2338e+00, -1.4201e+00,  ..., -4.1938e-01,
          8.6472e-01,  8.7383e-01],
        [-1.1779e+00,  3.4430e-01, -2.0729e+00,  ...,  1.4587e+00,
          1.1187e+00,  2.0780e+00]], grad_fn=<AddmmBackward0>)

In [ ]:
sample_input